<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Project_MLDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Politecnico di Torino**

**01TXFSM - Machine learning and Deep learning**

**Incremental Learning in Image Classification**

**Cordaro Nicolò - s272145**

**Di Nepi Marco - s277959**

**Falletta Alberto - s277971**


In [0]:
# !pip3 install 'torch==1.4.0'
# !pip3 install 'torchvision==0.5.0'
# !pip3 install 'Pillow-SIMD'
# !pip3 install 'tqdm'

**Imports**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn


import torchvision
from torchvision import transforms
from torchvision.models import resnet18

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import drive

**Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Init at 10 because first train is on 10 classes
NUM_CLASSES = 10

# Used for the pseudorandom shuffle of the split
SEED = 42

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.01            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 10      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Pre-processing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(224),      # Resizes short size of the PIL image to 256
                                      #transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # https://github.com/pytorch/examples/blob/master/imagenet/main.py
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(224),
                                      #transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))                                 
])

**Prepare Dataset**

CIFAR100 has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another.

Each of the downloaded files is a Python "pickled" object produced with cPickle.

In [4]:
# Clone github repository with data
# if os.path.isdir('./Project_MLDL'):
!rm -rf Project_MLDL
if not os.path.isdir('./CIFAR100_tError'):
  !git clone https://github.com/Nicordaro/Project_MLDL


Cloning into 'Project_MLDL'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 98 (delta 41), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [0]:
from Project_MLDL.CIFAR100_tError import CIFAR100_tError
import numpy as np
import random

DATA_DIR = './CIFAR100'

lbls = [i for i in range(0,100)]
random.seed(SEED)
random.shuffle(lbls)

added_labels=[]

def make_data_labels(lbls, added_labels):
  new_labels=[]
  for el in lbls[:10]:
    added_labels.append(el)
    new_labels.append(el)
  lbls = lbls[10:]

  return added_labels, lbls, new_labels

data -- a 50000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

labels -- a list of 50000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

**Create dataset**

In [7]:
# if not os.path.isdir('./CIFAR100'):

#le = preprocessing.LabelEncoder()

train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)
test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)
#FOR DEBUGGING PURPOSES
# lista = [1, 9, 15, 41, 42, 50, 65, 70, 78, 91]
# lista2 = [7, 14, 10, 24, 72, 93, 92, 35, 79, 80]
#le.fit([i for i in range(0,100)])

# ord_list = le.fit_transform(lista)
# print(ord_list)

# lista_tot = lista + lista2
# lista_tot_enc = le.fit_transform(lista_tot)
# print(lista_tot_enc)

# train_dataset.increment(lista,[i for i in range(0,10)])
# test_dataset.increment(lista,[i for i in range(0,10)])

Files already downloaded and verified
Files already downloaded and verified


In [0]:
train_dataset.increment(lista2,[i for i in range(10,20)])
test_dataset.increment(lista2,[i for i in range(10,20)])

**Dataloaders**

In [11]:
# added_labels, lbls, new_labels = make_data_labels(lbls, added_labels)
# train_dataset.increment(new_labels) #load 10 classes
# test_dataset.increment(new_labels)
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
lbl_train = []
for el in train_dataset:
  lbl_train.append(el[1])
print(np.unique(lbl_train))
lbl_test =[]
for el in test_dataset:
  lbl_test.append(el[1])
print(np.unique(lbl_test))
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Train dataset size: 10000
Test dataset size: 2000


**Prepare Network**

In [0]:
net = resnet18(pretrained=True)
net.fc = nn.Linear(512, NUM_CLASSES)

**Prepare Training**

In [0]:
# Loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Parameters to optimize:
parameters_to_optimize = net.parameters()

# Optimizer
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [0]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)
    
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step()

In [0]:
#test phase
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

In [0]:
test_dataset = CIFAR100_tError(DATA_DIR, train=False, transform=eval_transform, download=True)
for i in range(0,10): #one iteration for 10 classes
  print(f'Starting training with batch {i+1}')
  added_labels, lbls, new_labels = make_data_labels(lbls, added_labels)
  train_dataset = CIFAR100_tError(DATA_DIR, train=True, transform=train_transform, download=True)
  train_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])
  test_dataset.increment(new_labels,[j for j in range(0+i*10,10+i*10)])
  net.fc = nn.Linear(512, NUM_CLASSES*(i+1))
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

  #training 
  # By default, everything is loaded to cpu
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

  cudnn.benchmark # Calling this optimizes runtime

  current_step = 0

  # Start iterating over the epochs
  for epoch in range(NUM_EPOCHS):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
    
    # Iterate over the dataset
    for images, labels in train_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      net.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = net(images)

      # Compute loss based on output and ground truth
      loss = criterion(outputs, labels)
      
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1

    # Step the scheduler
    scheduler.step()

  #test phase
  net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataset))

  print('Test Accuracy: {}'.format(accuracy))

Files already downloaded and verified
Starting training with batch 1
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss 2.5131778717041016
Step 10, Loss 1.5857771635055542
Step 20, Loss 1.0479189157485962
Step 30, Loss 0.6465529799461365
Starting epoch 2/10, LR = [0.01]
Step 40, Loss 0.4985538423061371
Step 50, Loss 0.3380873203277588
Step 60, Loss 0.3569357991218567
Step 70, Loss 0.2864399552345276
Starting epoch 3/10, LR = [0.01]
Step 80, Loss 0.22897174954414368
Step 90, Loss 0.1876455843448639
Step 100, Loss 0.22799287736415863
Step 110, Loss 0.19851930439472198
Starting epoch 4/10, LR = [0.01]
Step 120, Loss 0.11093045026063919
Step 130, Loss 0.09009033441543579
Step 140, Loss 0.095253586769104
Step 150, Loss 0.11390882730484009
Starting epoch 5/10, LR = [0.01]
Step 160, Loss 0.05271262675523758
Step 170, Loss 0.08326221257448196
Step 180, Loss 0.06337614357471466
Step 190, Loss 0.06747893989086151
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 0.03614041209220886
Step 210, Loss 0.029769469052553177
Step 220, Loss 0.023922603577375412
Step 230, Loss 0.03677593171596527
Starting epoch 

100%|██████████| 8/8 [00:03<00:00,  2.58it/s]


Test Accuracy: 0.909
Starting training with batch 2
Files already downloaded and verified
Starting epoch 1/10, LR = [0.01]
Step 0, Loss 2.9670214653015137
Step 10, Loss 1.7291547060012817
Step 20, Loss 0.8575284481048584
Step 30, Loss 0.7054340839385986
Starting epoch 2/10, LR = [0.01]
Step 40, Loss 0.4638502895832062
Step 50, Loss 0.3027622699737549
Step 60, Loss 0.3704189956188202
Step 70, Loss 0.31771668791770935
Starting epoch 3/10, LR = [0.01]
Step 80, Loss 0.17669165134429932
Step 90, Loss 0.1888006031513214
Step 100, Loss 0.15467165410518646
Step 110, Loss 0.16878122091293335
Starting epoch 4/10, LR = [0.01]
Step 120, Loss 0.11024244874715805
Step 130, Loss 0.08977004885673523
Step 140, Loss 0.15614861249923706
Step 150, Loss 0.07679255306720734
Starting epoch 5/10, LR = [0.01]
Step 160, Loss 0.04520918428897858
Step 170, Loss 0.06010031700134277
Step 180, Loss 0.045991867780685425
Step 190, Loss 0.041909247636795044
Starting epoch 6/10, LR = [0.01]
Step 200, Loss 0.028886521235

100%|██████████| 16/16 [00:05<00:00,  2.73it/s]


Test Accuracy: 0.466
Starting training with batch 3
Files already downloaded and verified
Starting epoch 1/10, LR = [0.0001]
Step 0, Loss 3.3284244537353516
Step 10, Loss 3.245629072189331
Step 20, Loss 2.8266608715057373
Step 30, Loss 2.3995261192321777
Starting epoch 2/10, LR = [0.001]
Step 40, Loss 2.249440908432007
Step 50, Loss 1.950169324874878
Step 60, Loss 1.924938440322876
Step 70, Loss 1.6203266382217407
Starting epoch 3/10, LR = [0.001]
Step 80, Loss 1.3667521476745605
Step 90, Loss 1.3650256395339966
